Cryptocurrency trading bot.

In [1]:
##%pip install python-binance==0.7.5
#%pip install python-binance

## Global constants.

In [2]:
symbol = 'OCEANUSDT'
interval = '1h'
period = 100

## API key management.

In [3]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
client

## Get all available pairs for information needed to trade.

In [4]:
from cryptocurrency.exchange import Cryptocurrency_exchange
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info
exchange_info

,symbol,baseAsset,baseAssetPrecision,quoteAsset,quotePrecision,quoteAssetPrecision,baseCommissionPrecision,quoteCommissionPrecision,allowTrailingStop,cancelReplaceAllowed,min_price,max_price,tick_size,step_size,multiplier_up,multiplier_down
0,ETHBTC,ETH,8,BTC,8,8,8,8,True,True,1.000000e-06,922327.0,1.000000e-06,0.00010,5.0,0.2
1,LTCBTC,LTC,8,BTC,8,8,8,8,True,True,1.000000e-06,100000.0,1.000000e-06,0.00100,5.0,0.2
2,BNBBTC,BNB,8,BTC,8,8,8,8,True,True,1.000000e-06,100000.0,1.000000e-06,0.00100,5.0,0.2
3,NEOBTC,NEO,8,BTC,8,8,8,8,True,True,1.000000e-06,100000.0,1.000000e-06,0.01000,5.0,0.2
4,QTUMETH,QTUM,8,ETH,8,8,8,8,True,True,1.000000e-06,1000.0,1.000000e-06,0.10000,5.0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1440,GTOBUSD,GTO,8,BUSD,8,8,8,8,True,True,1.000000e-05,1000.0,1.000000e-05,0.10000,5.0,0.2
1441,SNXETH,SNX,8,ETH,8,8,8,8,True,True,1.000000e-06,1000.0,1.000000e-06,0.01000,5.0,0.2
1442,WBTCBUSD,WBTC,8,BUSD,8,8,8,8,True,True,1.000000e-02,1000000.0,1.000000e-02,0.00001,5.0,0.2
1443,BELETH,BEL,8,ETH,8,8,8,8,True,True,1.000000e-07,1000.0,1.000000e-07,0.10000,5.0,0.2


## Get conversion table for price conversions.

In [5]:
import pandas as pd

def get_conversion_table(client, exchange_info=exchange_info):
    conversion_table = pd.DataFrame(client.get_ticker())
    conversion_table = conversion_table[conversion_table['symbol'].isin(exchange_info['symbol'])].reset_index(drop=True)
    conversion_table[['priceChangePercent', 'lastPrice', 'volume', 'bidPrice', 'askPrice', 'bidQty', 'askQty', 'count']] = \
        conversion_table[['priceChangePercent', 'lastPrice', 'volume', 'bidPrice', 'askPrice', 'bidQty', 'askQty', 'count']].astype(float)
    return conversion_table

conversion_table = get_conversion_table(client=client, exchange_info=exchange_info)
conversion_table

,symbol,priceChange,priceChangePercent,weightedAvgPrice,prevClosePrice,lastPrice,lastQty,bidPrice,bidQty,askPrice,...,openPrice,highPrice,lowPrice,volume,quoteVolume,openTime,closeTime,firstId,lastId,count
0,ETHBTC,-0.00023400,-0.416,0.05630711,0.05621800,0.055988,34.19810000,0.055987,27.53930,0.055988,...,0.05622200,0.05674500,0.05571400,9.924680e+04,5588.30099484,1657340676170,1657427076170,354438994,354560030,121037.0
1,LTCBTC,0.00018500,7.708,0.00247673,0.00240100,0.002585,2.47100000,0.002584,39.71200,0.002585,...,0.00240000,0.00258600,0.00240000,9.618828e+04,238.23264390,1657340676550,1657427076550,81790106,81813271,23166.0
2,BNBBTC,-0.00007800,-0.696,0.01122059,0.01121300,0.011135,0.12200000,0.011134,3.14100,0.011135,...,0.01121300,0.01132800,0.01105000,4.078603e+04,457.64316466,1657340675881,1657427075881,190093859,190140612,46754.0
3,NEOBTC,-0.00000300,-0.693,0.00043581,0.00043300,0.000430,0.27000000,0.000429,622.02000,0.000430,...,0.00043300,0.00044500,0.00042700,3.055519e+04,13.31625342,1657340674899,1657427074899,43646927,43650837,3911.0
4,QTUMETH,-0.00004700,-1.847,0.00250476,0.00253000,0.002497,3.60000000,0.002498,189.90000,0.002501,...,0.00254400,0.00254700,0.00246600,2.097800e+03,5.25449210,1657340632559,1657427032559,5298538,5298750,213.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1440,GTOBUSD,-0.00018000,-0.861,0.02133773,0.02105000,0.020720,47917.80000000,0.020700,9661.80000,0.020880,...,0.02090000,0.02195000,0.02067000,7.066192e+06,150776.51776900,1657340650301,1657427050301,66747,67498,752.0
1441,SNXETH,-0.00002500,-1.190,0.00210972,0.00204900,0.002076,11.50000000,0.002076,201.91000,0.002088,...,0.00210100,0.00219000,0.00205600,5.236610e+03,11.04776448,1657340656747,1657427056747,2134,2243,110.0
1442,WBTCBUSD,-329.98000000,-1.535,21613.18155663,21549.77000000,21170.020000,0.01219000,21181.680000,0.00094,21211.300000,...,21500.00000000,21930.00000000,21100.00000000,4.978173e+01,1075941.56869330,1657340659029,1657427059029,8909,10529,1621.0
1443,BELETH,-0.00000300,-0.421,0.00071718,0.00071260,0.000710,11.20000000,0.000708,592.00000,0.000711,...,0.00071280,0.00074300,0.00070140,1.406272e+05,100.85452598,1657340653574,1657427053574,5094,6131,1038.0


## Pre-screen buyable base assets in pairs.

In [6]:
def get_tradable_tickers_info(client=client, conversion_table=conversion_table):
    conversion_table = conversion_table.copy()
    conversion_table = conversion_table[conversion_table['volume'] > 500]
    conversion_table['bidAskChangePercent'] = \
        (conversion_table['askPrice'] - conversion_table['bidPrice']) / conversion_table['askPrice']
    conversion_table['bidAskQtyChangePercent'] = \
        (conversion_table['askQty'] - conversion_table['bidQty']) / conversion_table['bidQty']
    conversion_table['bidAskChangePercent'] = conversion_table['bidAskChangePercent'].apply(abs)
    conversion_table[['bidAskChangePercent', 'bidAskQtyChangePercent']] *= 100
    conversion_table = conversion_table.fillna(0)
    conversion_table = conversion_table[conversion_table['bidAskChangePercent'] < 0.1]
    conversion_table = conversion_table[conversion_table['bidAskQtyChangePercent'] > 0.0]
    conversion_table['volumeMarketPercent'] = \
        conversion_table['volume'] / conversion_table['volume'].sum()
    conversion_table = conversion_table.sort_values(by='volumeMarketPercent', ascending=False)
    conversion_table = conversion_table[['symbol', 'priceChangePercent', 'lastPrice', 
                                         'volume', 'bidAskQtyChangePercent', 'count']]
    conversion_table = conversion_table.reset_index(drop=True)
    return conversion_table

def get_new_tickers(client=client, conversion_table=conversion_table):
    tickers = get_tradable_tickers_info(client=client, conversion_table=conversion_table)
    tickers_list = list(tickers['symbol'].unique())
    return tickers_list

tickers_list = get_new_tickers(client=client, conversion_table=conversion_table)
len(tickers_list)

113

## Order book.

In [7]:
from time import sleep, time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def get_order_book_depth(symbol):
    sleep(2.9)
    depth = client.get_order_book(symbol=symbol, limit=5000)
    frames = {side: pd.DataFrame(data=depth[side], columns=['price', 'quantity'], dtype=float) 
              for side in ['bids', 'asks']}
    frames_list = [frames[side].assign(side=side) for side in frames]
    data = pd.concat(frames_list, axis='index', ignore_index=True, sort=True)
    return data

def get_bid_ask_range(symbol):
    data = get_order_book_depth(symbol=symbol)
    min_prices = data.groupby('side').price.min()
    max_prices = data.groupby('side').price.max()
    center_price = ((min_prices.loc['asks'] - max_prices.loc['bids']) + max_prices.loc['bids'])
    bid_range = (((center_price - min_prices.loc['bids']) / min_prices.loc['bids']) * 100)
    ask_range = (((max_prices.loc['asks'] - center_price) / center_price) * 100)
    return bid_range, ask_range

def get_upper_range_bigger_than_threshold_percent(symbol, threshold=100):
    bid_range, ask_range = get_bid_ask_range(symbol)
    trigger = ask_range > threshold
    return trigger

def get_lower_range_bigger_than_threshold_percent(symbol, threshold=100):
    bid_range, ask_range = get_bid_ask_range(symbol)
    trigger = bid_range > threshold
    return trigger

get_upper_range_bigger_than_threshold_percent(symbol='USTCBUSD', threshold=100)

False

In [8]:
from tqdm import tqdm

while True:
    tickers_list = get_new_tickers(client=client, conversion_table=conversion_table)
    sleep(5)
    interesting_pairs = [symbol for symbol in tqdm(tickers_list) 
                         if get_upper_range_bigger_than_threshold_percent(symbol=symbol, threshold=10000)]
    if len(interesting_pairs) > 0:
        print(interesting_pairs)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 113/113 [05:52<00:00,  3.12s/it]


['JASMYUSDT', 'IOSTUSDT', 'ANKRUSDT', 'TRXBUSD', 'OXTUSDT', 'FIDABTC', 'ALGOBTC', 'ADAETH', 'ADABNB', 'WAVESBTC', 'ORNUSDT', 'KNCBTC', 'ATOMBTC', 'LINKETH', 'BNBBTC', 'BNBETH', 'ZECBTC', 'DASHBTC', 'XMRBNB']


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 113/113 [05:50<00:00,  3.11s/it]


['JASMYUSDT', 'IOSTUSDT', 'ANKRUSDT', 'TRXBUSD', 'OXTUSDT', 'FIDABTC', 'ALGOBTC', 'ADAETH', 'ADABNB', 'WAVESBTC', 'ORNUSDT', 'KNCBTC', 'ATOMBTC', 'LINKETH', 'BNBBTC', 'BNBETH', 'ZECBTC', 'DASHBTC', 'XMRBNB']


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 113/113 [05:51<00:00,  3.11s/it]


...

['JASMYUSDT', 'IOSTUSDT', 'ANKRUSDT', 'TRXBUSD', 'OXTUSDT', 'FIDABTC', 'ALGOBTC', 'ADAETH', 'ADABNB', 'WAVESBTC', 'ORNUSDT', 'KNCBTC', 'ATOMBTC', 'LINKETH', 'BNBBTC', 'BNBETH', 'ZECBTC', 'DASHBTC', 'XMRBNB']


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 113/113 [05:57<00:00,  3.16s/it]


['JASMYUSDT', 'IOSTUSDT', 'ANKRUSDT', 'TRXBUSD', 'OXTUSDT', 'FIDABTC', 'ALGOBTC', 'ADAETH', 'ADABNB', 'WAVESBTC', 'ORNUSDT', 'KNCBTC', 'ATOMBTC', 'LINKETH', 'BNBBTC', 'BNBETH', 'ZECBTC', 'DASHBTC', 'XMRBNB']


 15%|███████████████████▌                                                                                                              | 17/113 [00:53<05:06,  3.19s/it]

ReadTimeout: HTTPSConnectionPool(host='api.binance.com', port=443): Read timed out. (read timeout=10)

In [ ]:
tickers_list

In [ ]:
from time import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

symbol = 'USTCBUSD'
t1 = time()
depth = client.get_order_book(symbol=symbol, limit=5000)
t2 = time()
print('Elapsed time:', (t2 - t1), 'seconds')
frames = {side: pd.DataFrame(data=depth[side], columns=['price', 'quantity'], dtype=float) 
          for side in ['bids', 'asks']}
frames_list = [frames[side].assign(side=side) for side in frames]
data = pd.concat(frames_list, axis='index', ignore_index=True, sort=True)
t2 = time()
print('Elapsed time:', (t2 - t1), 'seconds')
max_quantities = data.groupby('side').quantity.max()
height_based_direction = ((max_quantities.loc['asks'] - max_quantities.loc['bids']) / max_quantities.loc['bids'])
height_based_direction *= 100
height_based_direction

In [ ]:
max_quantities.loc['bids'], max_quantities.loc['asks']

In [ ]:
data[data['side'] == 'bids'].shape[0], data[data['side'] == 'asks'].shape[0]

In [ ]:
min_prices = data.groupby('side').price.min()
max_prices = data.groupby('side').price.max()
asks_width = ((max_prices.loc['asks'] - min_prices.loc['asks']) / max_prices.loc['asks'])
bids_width = ((max_prices.loc['bids'] - min_prices.loc['bids']) / max_prices.loc['asks'])
width_based_direction = ((asks_width / (asks_width + bids_width)) * 100)
width_based_direction

In [ ]:
center_price = ((min_prices.loc['asks'] - max_prices.loc['bids']) + max_prices.loc['bids'])
bid_range = (((center_price - min_prices.loc['bids']) / min_prices.loc['bids']) * 100)
ask_range = (((max_prices.loc['asks'] - center_price) / center_price) * 100)
bid_range, ask_range

In [ ]:
depth_bullishness = height_based_direction + width_based_direction
depth_bullishness

In [ ]:
fig, ax = plt.subplots()

sns.ecdfplot(x='price', weights='quantity', stat='count', complementary=True, data=frames['bids'], ax=ax)
sns.ecdfplot(x='price', weights='quantity', stat='count', data=frames['asks'], ax=ax)
sns.scatterplot(x='price', y='quantity', hue='side', data=data, ax=ax)

ax.set_xlabel('Price')
ax.set_ylabel('Quantity')

plt.show()

## Smaller bid/ask split (no depth).

In [ ]:
df = pd.DataFrame(client.get_ticker())
split = df[df['symbol'] == symbol][['bidQty', 'askQty']].iloc[0].astype(float)
#(((split.loc['askQty'] - split.loc['bidQty']) / split.loc['bidQty']) * 100)
split